In [4]:
import findspark
findspark.init('/home/aman/spark-2.4.5-bin-hadoop2.7')

In [7]:
import pyspark
import copy
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql import functions as sf

spark = SparkSession.builder.getOrCreate()

df = spark.read.csv('Clean_post_data.csv', header=True,inferSchema=True)


In [8]:
df3 = df

In [9]:
df3.show(3)

+---+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|_c0| ID|                Body|               Title|                Tags|Score|                Code|            Keywords|
+---+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|  0|  4|want use trackbar...|convert decimal t...| c#  floating-poi...|  658| track-bar   form...|trackbar change f...|
|  1|  6|absolutely positi...|percentage width ...| html  css  inter...|  293| div   div   perc...|percentagebased w...|
|  2|  9|given datetime re...|how do i calculat...| c#  .net  datetime | 1819|           datetime |datetime represen...|
+---+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
only showing top 3 rows



In [10]:
df=df.drop('_c0')

In [11]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+----+-----+----+-----+----+--------+
| ID|Body|Title|Tags|Score|Code|Keywords|
+---+----+-----+----+-----+----+--------+
|  0|   0|    0|   0|    0|5279|      55|
+---+----+-----+----+-----+----+--------+



In [12]:
df = df.dropna()

In [13]:
df.show(3)

+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
| ID|                Body|               Title|                Tags|Score|                Code|            Keywords|
+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|  4|want use trackbar...|convert decimal t...| c#  floating-poi...|  658| track-bar   form...|trackbar change f...|
|  6|absolutely positi...|percentage width ...| html  css  inter...|  293| div   div   perc...|percentagebased w...|
|  9|given datetime re...|how do i calculat...| c#  .net  datetime | 1819|           datetime |datetime represen...|
+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
only showing top 3 rows



In [14]:

df= df.withColumn('Text', sf.concat(sf.col('Tags'),sf.lit(' '), sf.col('Body'))) 

In [15]:
df = df.select('Text','Tags')

In [16]:
df.show(3)


+--------------------+--------------------+
|                Text|                Tags|
+--------------------+--------------------+
| c#  floating-poi...| c#  floating-poi...|
| html  css  inter...| html  css  inter...|
| c#  .net  dateti...| c#  .net  datetime |
+--------------------+--------------------+
only showing top 3 rows



In [17]:
#df = df.dropna()
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+----+
|Text|Tags|
+----+----+
|   0|   0|
+----+----+



In [18]:
rowarr = df.select('Text').collect()

arr= [row.Text for row in rowarr]

IllegalArgumentException: 'Unsupported class file major version 55'

In [ ]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_df = tfidf_vectorizer.fit_transform( arr) #fitting and transforming the vector
tfidf_df

In [ ]:
#df2['queries'] = df2.union(queries)

#df2 = df2.withColumn('queries',sf.lit('php'))
df2 = spark.createDataFrame([{"queries": "php"}])

In [ ]:
df2.select('queries').collect()

In [ ]:
rowarr2 = df2.select('queries').collect()

arr2= [row.queries for row in rowarr2]

In [ ]:
user_tfidf = tfidf_vectorizer.transform(arr2)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_df)

In [ ]:
output2 = list(cos_similarity_tfidf)

In [ ]:
def get_recommendation(top, df3, scores):
    recommendation = False
    count = 0
    for i in top:
        
        body = df3.select('Body').collect()[i].Body
        tag = df3.select('Tags').collect()[i].Tags
        title = df3.select('Title').collect()[i].Title
        score = scores[count]
        sample = spark.createDataFrame([(body, tag, title, str(score))], ['Body', 'Tags', 'Title', 'Score'])
        if recommendation:
            
            recommendation = recommendation.union(sample)
        else:
            
            recommendation = sample
        count = count+1
    return recommendation.show()

In [ ]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]

top

In [ ]:
list_scores = [output2[i][0][0] for i in top]

list_scores

In [ ]:
get_recommendation(top,df3, list_scores)

The top recommendations using CountVectorize

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

count_df = count_vectorizer.fit_transform(arr) #fitting and transforming the vector
count_df

In [ ]:
query_count = count_vectorizer.transform(arr2)
cos_similarity_countv = map(lambda x: cosine_similarity(query_count, x),count_df)

In [ ]:
output2 = list(cos_similarity_countv)

In [ ]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]

In [ ]:
list_scores = [output2[i][0][0] for i in top]

In [ ]:
get_recommendation(top,df3, list_scores)

In [ ]:
from sklearn.neighbors import NearestNeighbors
n_neighbors = 10
KNN = NearestNeighbors(n_neighbors, p=2)
KNN.fit(tfidf_df)
NNs = KNN.kneighbors(user_tfidf, return_distance=True)

In [ ]:
NNs[0][0][1:]

In [ ]:
top = NNs[1][0][1:]
index_score = NNs[0][0][1:]

get_recommendation(top, df3, index_score)

WordCloud

In [ ]:

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
# Introducing index col to find out last n value

df = df.withColumn("index", monotonically_increasing_id())

In [ ]:
last_value_row_arr = df.select('Text').where(sf.col('index')>df.count()-1000).collect()

last_value = [row.Text for row in last_value_row_arr]

In [ ]:
bunch_text = " ".join(text for text in last_value)

In [ ]:
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color="white", colormap= "magma").generate(bunch_text)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[11,11])
plt.imshow(wordcloud, interpolation="sinc")
plt.axis("off")
plt.show()